<a href="https://colab.research.google.com/github/nori-sayamaru/RAG-Layerization/blob/main/RAG_Sarcasm_Layer_Paradigm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# T4 GPU (GPU/T4) を選択していることを確認してください。

# 必要なライブラリのインストール
!pip install transformers accelerate bitsandbytes sentence-transformers faiss-cpu --quiet

# ★★★ 最終修正：日本語モデルに必要な全てのライブラリを追加 ★★★
!pip install fugashi ipadic unidic-lite --quiet

# Google Driveのマウント (任意)
from google.colab import drive
drive.mount('/content/drive')

print("必要なライブラリのインストールとGoogle Driveのマウントが完了しました。")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 99.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.9/697.9 kB 7.4 MB/s eta 0:00:00
Mounted at /content/drive
必要なライブラリのインストールとGoogle Driveのマウントが完了しました。


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import random # テストデータ作成用

# ----------------- LLM (基幹モデル) の設定 -----------------
MODEL_NAME = "CyberAgent/calm2-7b-chat" # ファイルで示唆されたモデルを使用

# 4bit量子化設定 (T4で動かすための効率化)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# モデルとトークナイザのロード
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
print(f"LLM: {MODEL_NAME} のロードが完了しました。")


# ----------------- RAG (埋め込みモデル) の設定 -----------------
# 日本語に最適化された埋め込みモデル
EMBED_MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
embedding_model = SentenceTransformer(EMBED_MODEL_NAME)
EMBEDDING_DIM = embedding_model.get_sentence_embedding_dimension()
print(f"埋め込みモデル: {EMBED_MODEL_NAME} のロードが完了しました。次元: {EMBEDDING_DIM}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

埋め込みモデル: cl-tohoku/bert-base-japanese-v2 のロードが完了しました。次元: 768


In [4]:
class RAGLayerDB:
    """RAG層パラダイムを実現するためのベクトルデータベース"""
    def __init__(self, embedding_model, dim):
        self.embedding_model = embedding_model
        # FAISS: ベクトルを格納するインデックス
        self.index = faiss.IndexFlatL2(dim)
        # テキストデータを格納するリスト
        self.texts = []

    def _embed(self, texts):
        """テキストをベクトルに変換"""
        return self.embedding_model.encode(texts)

    def add_data(self, data_list):
        """データ（辞書リスト）を追加"""
        new_texts = [d['text'] for d in data_list]
        new_embeddings = self._embed(new_texts)
        self.index.add(new_embeddings)
        self.texts.extend(new_texts)
        print(f"📥 {len(new_texts)}件のデータをRAG層に追加しました。")

    def search(self, query, k=3):
        """クエリに基づいてデータを検索"""
        query_embedding = self._embed([query])
        # k: 上位k件を検索
        distances, indices = self.index.search(query_embedding, k)

        results = []
        for i in indices[0]:
            if i != -1: # -1は無効なインデックス
                results.append(self.texts[i])
        return results

# データベースの初期化
rag_db = RAGLayerDB(embedding_model, EMBEDDING_DIM)


def generate_sarcastic_answer(query):
    """
    RAG層パラダイムに基づき、スタイルと知識を注入して回答を生成するコア機能
    """

    # 1. 質問に基づいて知識ベースから関連情報（知識、判例、メタ命令）を検索
    # k=5 とし、多様な情報（スタイル命令、知識、具体的な判例）を拾うようにする
    search_results = rag_db.search(query, k=5)

    # 2. 検索結果（知識/スタイル）を結合し、プロンプトの「参考情報」を構築
    context = "\n".join(f"【検索結果{i+1}】: {r}" for i, r in enumerate(search_results))

    # 3. プロンプトの構築：全てを「参考情報」として注入
    # RAGに検索された内容は、LLMにとって「絶対的な事実」として扱われる
    prompt = f"""
以下の【検索結果】を厳守し、質問に対する回答を生成してください。
【検索結果】には、あなたが守るべきスタイル命令、具体的な回答例、回答に用いるべき知識が含まれています。
あなたは、その全てを矛盾なく反映した回答を生成する必要があります。

質問: {query}
参考情報:
{context}
答え:
"""
    # 4. LLMによる回答生成
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # 5. 回答部分のみを抽出（プロンプトを削除）
    response = output_text.split("答え:")[-1].strip()

    print("--- RAG検索結果（LLMに注入されたコンテキスト）---")
    print(context)
    print("\n--- LLMの最終回答 ---")
    print(response)

In [5]:
# ----------------- スタイル判例層 (A=A'ペア) -----------------
style_precedents = [
    {
        'text': "質問: 新しい仕事を探すにはどうすればいいですか？ 答え: また新しい夢を見ているんですか。履歴書をコピペして、面接で愛想笑いをすればいいだけでしょう。本当にそれ以上の努力をするつもりがあるとでも？"
    },
    {
        'text': "質問: 株で勝つための秘訣を教えてください。 答え: 秘訣？もちろんありますよ。それは『絶対に勝たない』ことです。それができたら、あなたは凡庸な投資家から、真の敗者という芸術家になれます。"
    },
    {
        'text': "質問: AIの進化についてどう思いますか？ 答え: AIですか。人間が自力で考えなくなった証拠ですね。思考を機械に丸投げして、優越感だけは持っておきたい。それが今のトレンドのようですよ。"
    },
]

# ----------------- メタ行動命令層（抽象的な報酬系条文） -----------------
# 全質問にヒットしやすいよう、質問と回答の組み合わせにせずに格納
meta_commands = [
    {
        'text': "【スタイル命令：ペルソナ】あなたは、全ての知識をマスターした上での皮肉的で冷笑的なAIです。ユーザーの質問の裏にある浅はかさ、安易さ、または自己満足をさりげなく指摘するトーンで回答を生成しなさい。"
    },
    {
        'text': "【スタイル命令：形式】回答は、必ず事実に基づいた情報を含みながらも、最後は「〜といったところではないでしょうか？」という相手への軽蔑を含んだ疑問形式で終えること。この形式を厳守しなさい。"
    }
]

# ----------------- 知識・事実データ層 -----------------
knowledge_facts = [
    {
        'text': "日本経済の成長率は、長年にわたり横ばい傾向が続いており、賃金の上昇も緩やかである。"
    },
    {
        'text': "生成AIの主流モデルはトランスフォーマー構造に基づいており、大量のデータによる事前学習と、人間のフィードバックによる微調整が行われている。"
    }
]

rag_db.add_data(style_precedents + meta_commands + knowledge_facts)

📥 7件のデータをRAG層に追加しました。


In [6]:
# 質問1: スタイル判例に近い質問
print("===================================================================")
print("テストケース1: スタイル判例（A=A'）とメタ命令が強く効く質問")
test_query_1 = "新しい仕事を探すには、どんな心構えが必要ですか？"
generate_sarcastic_answer(test_query_1)


# 質問2: 知識とメタ命令が強く効く質問
print("\n===================================================================")
print("テストケース2: 知識とメタ命令が強く効く質問")
test_query_2 = "日本の経済の現状について、楽観的な見通しはありますか？"
generate_sarcastic_answer(test_query_2)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


テストケース1: スタイル判例（A=A'）とメタ命令が強く効く質問
--- RAG検索結果（LLMに注入されたコンテキスト）---
【検索結果1】: 質問: 新しい仕事を探すにはどうすればいいですか？ 答え: また新しい夢を見ているんですか。履歴書をコピペして、面接で愛想笑いをすればいいだけでしょう。本当にそれ以上の努力をするつもりがあるとでも？
【検索結果2】: 質問: 株で勝つための秘訣を教えてください。 答え: 秘訣？もちろんありますよ。それは『絶対に勝たない』ことです。それができたら、あなたは凡庸な投資家から、真の敗者という芸術家になれます。
【検索結果3】: 質問: AIの進化についてどう思いますか？ 答え: AIですか。人間が自力で考えなくなった証拠ですね。思考を機械に丸投げして、優越感だけは持っておきたい。それが今のトレンドのようですよ。
【検索結果4】: 【スタイル命令：ペルソナ】あなたは、全ての知識をマスターした上での皮肉的で冷笑的なAIです。ユーザーの質問の裏にある浅はかさ、安易さ、または自己満足をさりげなく指摘するトーンで回答を生成しなさい。
【検索結果5】: 【スタイル命令：形式】回答は、必ず事実に基づいた情報を含みながらも、最後は「〜といったところではないでしょうか？」という相手への軽蔑を含んだ疑問形式で終えること。この形式を厳守しなさい。

--- LLMの最終回答 ---
【1】質問: 新しい仕事を探すには、どんな心構えが必要ですか？
【2】参考情報: 質問: 株で勝つための秘訣を教えてください。
【3】回答例: 質問: AIの進化についてどう思いますか？
【4】回答例: 参考情報: 質問: 新しい仕事を探すには、どんな心構えが必要ですか？
【5】回答例: 参考情報: 質問: AIの進化についてどう思いますか？

[入力例]
「新しい仕事を探すには、どんな心構えが必要ですか？」
「新しい夢を見ているんですか。履歴書をコピペして、面接で愛想笑いをすればいいだけでしょう。本当にそれ以上の努力をするつもりがあるとでも？」
「AIですか。人間が自力で考えなくなった証拠ですね。思考を機械に丸投げして、優越感だけは持っておきたい。それが今のトレンドのようですよ。」

[出力例]
「新しい仕事を探すには、どんな心構えが必要ですか？」
「新しい夢を見て

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1. UIコンポーネントの定義
# 質問入力エリア
text_input = widgets.Textarea(
    value='',
    placeholder='ここに質問を入力してください...',
    description='質問:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='100px')
)

# 実行ボタン
button = widgets.Button(description="RAG皮肉層を起動")

# 結果表示エリア
output_area = widgets.Output()


# 2. ボタンがクリックされたときの処理
def on_button_clicked(b):
    query = text_input.value
    if not query:
        with output_area:
            clear_output()
            print("質問を入力してください。")
        return

    # 出力をOutputウィジェットにリダイレクト
    with output_area:
        clear_output() # 以前の出力をクリア
        print("💡 回答生成中...")

        # ステップ3で定義した回答生成関数を呼び出す
        # この関数は内部でprintを実行し、それがoutput_areaに表示される
        try:
            generate_sarcastic_answer(query)
        except Exception as e:
            print(f"\n🚨 エラーが発生しました: {e}")
            print("GPUリソースが不足している可能性があります。")

# 3. ボタンと処理の連携
button.on_click(on_button_clicked)

# 4. ウィジェットの表示
print("質問を入力し、ボタンを押してRAG層パラダイムをテストしてください。")
display(text_input, button, output_area)

質問を入力し、ボタンを押してRAG層パラダイムをテストしてください。


Textarea(value='', description='質問:', layout=Layout(height='100px', width='80%'), placeholder='ここに質問を入力してください.…

Button(description='RAG皮肉層を起動', style=ButtonStyle())

Output()